In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from tensorflow.examples.tutorials.mnist import input_data

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
tf.reset_default_graph()

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
LATENT_DIM = 120
H_DIM = 160
IMG_DIM = 784
X = tf.placeholder(tf.float32, shape=[None,IMG_DIM])
z = tf.placeholder(tf.float32, shape=[None,LATENT_DIM])

In [5]:
# Decoder
def P(z):
    with tf.name_scope("decoder"):
        with tf.variable_scope("decoder", initializer=tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE):
            x = tf.layers.dense(z, H_DIM, activation = tf.nn.relu)
            logits = tf.layers.dense(x, IMG_DIM)
            prob = tf.nn.sigmoid(logits)
        
    return logits, prob

In [6]:
# Encoder: output vectors of means and variances.
def Q(x):
    with tf.name_scope("encoder"):
        with tf.variable_scope("encoder", initializer=tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE):
            x = tf.layers.dense(x, H_DIM, activation=tf.nn.relu)
            mu = tf.layers.dense(x, LATENT_DIM)
            # I use a vector since we can represent a diagonal matrix as a vector.
            sigma = tf.layers.dense(x, LATENT_DIM)
        
    return mu, sigma

In [7]:
def sample_z(mu, sigma):
    eps = tf.random_normal(shape=tf.shape(mu))
    return mu + tf.exp(sigma / 2) * eps

In [8]:
# Losses
mu, sigma = Q(X)
z_sample = sample_z(mu,sigma)
logits, _ = P(z_sample)

# Sampling from random z
X_samples, _ = P(z)

# E[log P(X|z)]
recon_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=X), 1)
KL_loss =  0.5 * tf.reduce_sum(tf.exp(sigma) + mu**2 - 1. - sigma, 1)
loss = tf.reduce_mean(recon_loss + KL_loss)

solver = tf.train.AdamOptimizer().minimize(loss)

In [9]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('vae_out/'):
    os.makedirs('vae_out/')

i = 0
batch_size = 64

for it in range(1000000):
    X_mb, _ = mnist.train.next_batch(batch_size)

    _, vae_loss = sess.run([solver, loss], feed_dict={X: X_mb})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('Loss:')
        print(vae_loss)
        print()

        samples = sess.run(X_samples, feed_dict={z: np.random.randn(16, LATENT_DIM)})

        fig = plot(samples)
        plt.savefig('vae_out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

Iter: 0
Loss:
569.4079

Iter: 1000
Loss:
147.84387

Iter: 2000
Loss:
131.70375

Iter: 3000
Loss:
125.90325

Iter: 4000
Loss:
112.35951

Iter: 5000
Loss:
115.149826

Iter: 6000
Loss:
113.4042

Iter: 7000
Loss:
109.82806

Iter: 8000
Loss:
107.92288

Iter: 9000
Loss:
111.51605

Iter: 10000
Loss:
109.18927

Iter: 11000
Loss:
112.26142

Iter: 12000
Loss:
106.24942



KeyboardInterrupt: 